In [19]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_random_exponential
import os

import openai

from utils.embedding_utils import get_embedding, sliding_window

In [13]:
df = pd.read_csv("../data/db/df_text.csv")
df.head()

,source_type,text
0,paper,\begin{document}\n\n\preprint{MIT-CTP 5315}\n\...
1,paper,% ============================================...
2,paper,\begin{document}\n\n%\preprint{}\n\n\title{Pre...
3,paper,Once detector data has been processed and high...
4,paper,\begin{document}\n\preprint{MIT--CTP 5223}\n\n...


In [14]:
df_filtered = df  # [~df['source_location'].str.contains('hep')].sort_values(by=['source_location'])

In [15]:
window_size = 512
stride = 384

text_chunks = []
embeddings = []

for i in tqdm(range(len(df_filtered))):
    text = df_filtered['text'].values[i].replace('\n', ' ').strip()
    text_chunks_i = list(sliding_window(text, window_size, stride))
    if len(text_chunks_i) > 80:
        text_chunks_i = text_chunks_i[:80]
    embeddings_i = [get_embedding(text) for text in text_chunks_i]
    text_chunks += text_chunks_i
    embeddings += embeddings_i

100%|██████████| 57/57 [04:54<00:00,  5.16s/it]


In [16]:
embeddings = np.array(embeddings, dtype=np.float64)

In [17]:
data = [text_chunks, embeddings]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['text_chunks', 'embeddings']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

In [18]:
df.to_csv('../data/db/text_chunks.csv', index=False)
np.save('../data/db/embeddings.npy', embeddings)